In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("covid_toy.csv")

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
# without use of column transformer


In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop("has_covid",axis=1),df["has_covid"],test_size=0.2)

In [9]:
# simple imputer - filling null values
si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[["fever"]])
x_test_fever=si.fit_transform(x_test[["fever"]])

In [10]:
x_train.shape

(80, 5)

In [11]:
# ordinal encoder
oe=OrdinalEncoder()
x_train_cough=oe.fit_transform(x_train[["cough"]])
x_test_cough=oe.fit_transform(x_test[["cough"]])
x_train_cough.shape

(80, 1)

In [14]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [15]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [16]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

In [17]:
# with the use of column transformer

In [18]:
from sklearn.compose import ColumnTransformer

In [19]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),["fever"]),
    ('tnf2',OrdinalEncoder(categories=[["Mild","Strong"]]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [20]:
transformer.fit_transform(x_train).shape

(80, 7)

In [21]:
transformer.fit_transform(x_test).shape

(20, 7)